In [1]:
import ipywidgets as widgets
import seaborn as sns
from matplotlib import pyplot as plt
import json
import pandas as pd
from hockey_rink import NWHLRink
from IPython.display import display
from tabulate import tabulate

In [2]:

df = pd.read_csv('data/df.csv',low_memory=False)
df=df.sort_values('game_id')
df['seasonid'] = df['game_id'].astype(str).str[:4]
df['type'] = df['game_id'].astype(str).str[4:6]
df["gameid"] = df['game_id'].astype(str).str[6:10]
df_sorted = df.sort_values(by=['game_id'])

In [3]:
season_slider = widgets.SelectionSlider(
    options=df['seasonid'].unique(),
    value=df['seasonid'].min(),
    description='season',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True
)
type_slider = widgets.SelectionSlider(
    options=df['type'].unique(),
    value=df['type'].min(),
    description='type',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True
)
id_slider = widgets.SelectionSlider(
    options=df['gameid'].unique(),
    value=df['gameid'].min(),
    description='gameid',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    step=1,
    readout=True
)
df_sortednew=df_sorted.sort_values(by='event_idx')
df_sortednew.loc[(df_sortednew.event_type.isin(["SHOT", "GOAL"]))]
event_slider = widgets.SelectionSlider(
    
    options=df_sortednew['event_idx'].unique(),
    value=df_sortednew['event_idx'].min(),
    description='i',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True
)


In [4]:
display(season_slider)
display(type_slider)
display(id_slider)

SelectionSlider(continuous_update=False, description='season', options=('2016', '2017', '2018', '2019', '2020'…

SelectionSlider(continuous_update=False, description='type', options=('02', '03'), value='02')

SelectionSlider(continuous_update=False, description='gameid', options=('0001', '0002', '0003', '0004', '0005'…

In [5]:


def slid(season_id,Type_id,game_id,event):
    df_sorted = df[df['game_id'] == int(season_slider.value+type_slider.value+id_slider.value)]
    game_id=season_slider.value+type_slider.value+id_slider.value
    season=(season_slider.value)
    
    for ind in df_sorted.index: 
        if df_sorted['team_home'][ind] :
            home_team = df_sorted['team_tri_code'][ind]
        else:
            away_team = df_sorted['team_tri_code'][ind]
    if int(type_slider.value) == 2:
        types='regular'
    else:
        types='playoffs'
    gameid1=season_slider.value+type_slider.value+id_slider.value
    shootouts = df_sorted[df_sorted['period'] == 5]
    is_home = shootouts['team_tri_code']==home_team
    is_away = shootouts['team_tri_code']==away_team
    is_goal =shootouts['event_type']=='GOAL'
    is_shot =shootouts['event_type']=='SHOT'
    shootoutshomegoals = shootouts[(is_home) & (is_goal)]
    shootoutsawaygoals = shootouts[(is_away) & (is_goal)]
    shootoutshomeshots = shootouts[(is_home) & (is_shot)]
    shootoutsawayshots = shootouts[(is_away) & (is_shot)]
    
    f = open('data/'+season+'/'+types+'/'+gameid1+'.json')
    test_data = json.load(f)
    td_home = test_data["liveData"]["boxscore"]["teams"]["home"]["teamStats"]["teamSkaterStats"]
    td_away = test_data["liveData"]["boxscore"]["teams"]["away"]["teamStats"]["teamSkaterStats"]
    goals_away=td_away["goals"]
    shots_away=td_away["shots"]
    goals_home=td_home["goals"]
    shots_home=td_home["shots"]
    D = test_data["liveData"]["plays"]["allPlays"][event_slider.value]
    
    print(df_sorted['game_start_time'].iloc[0])
    print('Game ID:'+ df_sorted['gameid'].iloc[0]+' ; '+ home_team +  '(home) vs '+ away_team+'(away)')
    print(tabulate([['Teams:', home_team,away_team], ['Goals:', str(goals_home),str(goals_away)],['SoG:', str(shots_home),str(shots_away)], ['SO Goals:', len(shootoutshomegoals),len(shootoutsawaygoals)], ['SO Attempts:', len(shootoutshomeshots),len(shootoutsawayshots)]], headers=['', 'Home','Away']))
    for i in ('about','coordinates','players','result',"team"):
        display(i+':', D[i])
    x = str(event_slider.value)
    shots = df_sorted.loc[(df_sorted.event_type.isin(["SHOT", "GOAL"]))]
    shots= shots[shots['event_idx']==event_slider.value]
    home_shots = shots[shots.team_tri_code == home_team]
    away_shots = shots[shots.team_tri_code == away_team]

    rink = NWHLRink(x_shift=100, y_shift=42.5, nzone={"length": 50})
    ax = rink.draw()
    rink.scatter(home_shots["coordinates_x"].abs(), home_shots["coordinates_y"].abs())
    rink.scatter(200 - away_shots["coordinates_x"].abs(), 85 - away_shots["coordinates_y"].abs())

    return 

widgets.interact(slid,season_id=season_slider,Type_id=type_slider,game_id=id_slider,event=event_slider)




interactive(children=(SelectionSlider(continuous_update=False, description='season', options=('2016', '2017', …

<function __main__.slid(season_id, Type_id, game_id, event)>